## Instalação e Preparação do Ambiente

In [ ]:
!pip install duckdb
!pip install pandas
!pip install dotenv
!pip install -q -U google-genai

In [2]:
import os
import duckdb
import pandas as pd
import glob
import json

import google.generativeai as genai
from dotenv import load_dotenv

In [4]:
load_dotenv()

db_user = os.getenv("DB_USER")
db_password = os.getenv("DB_PASSWORD")
db_host = os.getenv("DB_HOST")
db_port = os.getenv("DB_PORT")
db_name = os.getenv("DB_NAME")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

## Extração dos Dados

In [ ]:
OPEN_STATUS = 'Aberto'
TOP_N_SUBCATEGORIES = 3
MAX_TICKETS_PER_COMPANY = 1000

conn_string = f"dbname={db_name} user={db_user} password={db_password} host={db_host} port={db_port}"

In [ ]:
try:
    con = duckdb.connect(database=':memory:')
    
    con.execute("INSTALL postgres;")
    con.execute("LOAD postgres;")
    
    con.execute(f"ATTACH $${conn_string}$$ AS postgres_db (TYPE POSTGRES);")
    
    print("DuckDB -> Postgres connection established successfully")
except Exception as e:
    print(f"Error: {e}")
    exit()

In [ ]:
query = f"""
WITH SubcategoryRanks AS (
    SELECT
        co.name AS company_name,
        p.name AS product_name,
        sc.name AS subcategory_name,
        ROW_NUMBER() OVER(PARTITION BY co.name, p.name ORDER BY COUNT(t.ticket_id) DESC) as rank_num
    FROM
        postgres_db.public.tickets AS t
    JOIN postgres_db.public.companies AS co ON t.company_id = co.company_id
    JOIN postgres_db.public.products AS p ON t.product_id = p.product_id
    JOIN postgres_db.public.subcategories AS sc ON t.subcategory_id = sc.subcategory_id
    JOIN postgres_db.public.statuses AS st ON t.current_status_id = st.status_id
    WHERE
        st.name = '{OPEN_STATUS}'
    GROUP BY
        company_name, product_name, subcategory_name
),

CompanyQuota AS (
    SELECT
        company_name,
        FLOOR({MAX_TICKETS_PER_COMPANY} / (COUNT(DISTINCT product_name) * {TOP_N_SUBCATEGORIES})) AS tickets_per_slot_quota
    FROM
        SubcategoryRanks
    WHERE
        rank_num <= {TOP_N_SUBCATEGORIES}
    GROUP BY
        company_name
),

RankedTickets AS (
    SELECT
        co.name AS company_name,
        p.name AS product_name,
        sc.name AS subcategory_name,
        t.title,
        t.description,
        st.name AS status_name,
        t.created_at,
        cq.tickets_per_slot_quota,
        ROW_NUMBER() OVER(PARTITION BY co.name, p.name, sc.name ORDER BY t.created_at DESC) as ticket_rank
    FROM
        postgres_db.public.tickets AS t
    JOIN postgres_db.public.companies AS co ON t.company_id = co.company_id
    JOIN postgres_db.public.products AS p ON t.product_id = p.product_id
    JOIN postgres_db.public.subcategories AS sc ON t.subcategory_id = sc.subcategory_id
    JOIN postgres_db.public.statuses AS st ON t.current_status_id = st.status_id
    JOIN SubcategoryRanks sr ON co.name = sr.company_name AND p.name = sr.product_name AND sc.name = sr.subcategory_name
    JOIN CompanyQuota cq ON co.name = cq.company_name
    WHERE
        sr.rank_num <= {TOP_N_SUBCATEGORIES}
        AND st.name = '{OPEN_STATUS}'
),

UniqueTickets AS (
    SELECT
        *,
        ROW_NUMBER() OVER(PARTITION BY title, description ORDER BY created_at DESC) as unique_rank
    FROM
        RankedTickets
    WHERE
        ticket_rank <= tickets_per_slot_quota
)

SELECT
    company_name,
    product_name,
    subcategory_name,
    title,
    description,
    status_name
FROM
    UniqueTickets
WHERE
    unique_rank = 1;
"""

In [ ]:
try:
    df_chamados = con.execute(query).fetchdf()
    print(f"Total of {len(df_chamados)} tickets found")

    if df_chamados.empty:
        print("No tickets found")
    else:
        companhias = df_chamados['company_name'].unique()

        output_dir = "../data/original_tickets"
        os.makedirs(output_dir, exist_ok=True)

        for comp in companhias:
            df_companhia = df_chamados[df_chamados['company_name'] == comp]
            
            safe_filename = str(comp).lower().replace(' ', '_').replace('/', '_') + ".csv"
            output_path = os.path.join(output_dir, safe_filename)
            
            df_companhia.to_csv(output_path, index=False, encoding='utf-8-sig')

except Exception as e:
    print(f"Error: {e}")
finally:
    con.close()

## Verificação dos Dados

In [ ]:
input_dir = "../data/original_tickets"

csv_pattern = os.path.join(input_dir, '*.csv')
csv_files = glob.glob(csv_pattern)

if not csv_files:
    exit()

all_dataframes = []

for file_path in csv_files:
    try:
        df = pd.read_csv(file_path)
        all_dataframes.append(df)
    except Exception as e:
        print(f"Error: {file_path}: {e}")

if not all_dataframes:
    exit()

master_df = pd.concat(all_dataframes, ignore_index=True)

required_columns = ['company_name', 'product_name', 'subcategory_name']
if not all(col in master_df.columns for col in required_columns):
    exit()

summary = master_df.groupby(['company_name', 'product_name', 'subcategory_name']).size().reset_index(name='total_chamados')


print("\n--- Ticket Report by Customer, Product, and Subcategory ---")
print(summary.to_string())

output_report_path = '../data/report_tickets/grouped_report_v4.csv'
try:
    summary.to_csv(output_report_path, index=False, encoding='utf-8-sig')
except Exception as e:
    print(f"\nError: {e}")

In [ ]:
output_dir = "../data/original_tickets"

all_dfs = []

if os.path.exists(output_dir):
    for filename in os.listdir(output_dir):
        if filename.endswith(".csv"):
            file_path = os.path.join(output_dir, filename)
            try:
                df = pd.read_csv(file_path)
                all_dfs.append(df)
            except Exception as e:
                print(f"Error reading file {filename}: {e}")

    if all_dfs:
        full_df = pd.concat(all_dfs, ignore_index=True)

        duplicated_titles = full_df[full_df.duplicated(subset=['title'], keep=False)]
        
        if not duplicated_titles.empty:
            print(f"Found {duplicated_titles.shape[0]} records with duplicate titles.")
            print(duplicated_titles[['title', 'company_name']].head())
        else:
            print("No duplicate titles found.")

        print("-" * 30)

        duplicated_descriptions = full_df[full_df.duplicated(subset=['description'], keep=False)]

        if not duplicated_descriptions.empty:
            print(f"Found {duplicated_descriptions.shape[0]} records with duplicate descriptions.")
            print(duplicated_descriptions[['description', 'company_name']].head())
        else:
            print("No duplicate descriptions found.")
    else:
        print("No CSV files found for analysis.")
else:
    print(f"Directory '{output_dir}' not found.")

## Inferência LLM

In [5]:
genai.configure(api_key=GOOGLE_API_KEY)

input_dir = "../data/original_tickets"

TOP_N_THEMES = 3

# TODO ACRESCENTAR STARTIME, ENDTIME E DTH

# 22 MINUTOS PARA RODAR TUDO - 2.5 FLASH

def preparar_contexto_do_dataframe(df_produto):
    """Prepares the raw text from the tickets for the LLM."""
    contexto_textual = ""
    for index, row in df_produto.iterrows():
        contexto_textual += f"--- Ticket {index + 1} (Subcategory: {row['subcategory_name']}) ---\n"
        contexto_textual += f"Title: {row['title']}\n"
        contexto_textual += f"Description: {row['description']}\n"
    return contexto_textual

def criar_prompt(contexto_chamados, company_name, product_name, subcategory_stats_str):
    """
    Creates a prompt with statistics and requests an output in the format of insights and actions.
    """
    prompt = f"""
    Você é um Product Manager, com foco em análise de dados para tomada de decisão.

    **Contexto:**
    Você está analisando os chamados de suporte para a empresa "{company_name}", focando no produto "{product_name}".
    Uma análise quantitativa prévia dos chamados revelou os seguintes temas (subcategorias) como os mais problemáticos:

    **Estatísticas dos Principais Problemas:**
    {subcategory_stats_str}

    **Sua Tarefa:**
    Com base nas estatísticas acima e nos detalhes dos chamados fornecidos abaixo, sua missão é gerar um plano de ação conciso. Para cada um dos temas principais:
    1.  Sintetize o problema central.
    2.  Proponha uma lista de ações claras, específicas e implementáveis para resolver a causa raiz desses problemas.

    **Formato da Resposta:**
    Sua resposta DEVE ser um objeto JSON válido, sem nenhum texto ou formatação adicional. A estrutura deve ser a seguinte:
    {{
      "company_name": "{company_name}",
      "product_name": "{product_name}",
      "insights": [
        {{
          "theme": "Nome do Tema Principal 1 (Subcategoria)",
          "percentage": "XX%",
          "actions": [
            "Ação sugerida 1 para o Tema 1.",
            "Ação sugerida 2 para o Tema 1."
          ]
        }},
        {{
          "theme": "Nome do Tema Principal 2 (Subcategoria)",
          "percentage": "YY%",
          "actions": [
            "Ação sugerida 1 para o Tema 2."
          ]
        }}
      ]
    }}

    **Dados Brutos dos Chamados para Análise Qualitativa:**
    {contexto_chamados}
    """
    return prompt


model = genai.GenerativeModel('gemini-2.5-flash')

csv_files = [f for f in os.listdir(input_dir) if f.endswith('.csv')]

if not csv_files:
    print(f"No CSV files found in the directory '{input_dir}'.")
else:
    for csv_file in csv_files:
        caminho_completo = os.path.join(input_dir, csv_file)
        
        try:
            df_cliente = pd.read_csv(caminho_completo)
            
            if df_cliente.empty:
                print("File is empty. Skipping.")
                continue

            company_name = df_cliente.iloc[0]['company_name']
            
            produtos_no_arquivo = df_cliente['product_name'].unique()
            print(f"   -> Found products: {list(produtos_no_arquivo)}")

            for product_name in produtos_no_arquivo:
                print(f"\n   >> Analyzing product: '{product_name}'...")
                
                df_produto_especifico = df_cliente[df_cliente['product_name'] == product_name]
                
                print(f"      -> Calculating statistics with Pandas...")
                subcategory_dist = df_produto_especifico['subcategory_name'].value_counts(normalize=True)
                
                top_themes = subcategory_dist.head(TOP_N_THEMES)
                
                stats_str = ""
                for theme, percentage in top_themes.items():
                    stats_str += f"- **{theme}:** {percentage:.0%} dos chamados\n"
                
                print(f"      -> Statistics found:\n{stats_str}")

                contexto = preparar_contexto_do_dataframe(df_produto_especifico)
                
                if contexto:
                    prompt_final = criar_prompt(contexto, company_name, product_name, stats_str)
                    
                    try:
                        response = model.generate_content(prompt_final)
                        
                        cleaned_response = response.text.strip()
                        json_start = cleaned_response.find('{')
                        json_end = cleaned_response.rfind('}') + 1
                        
                        if json_start != -1 and json_end != -1:
                            json_str = cleaned_response[json_start:json_end]
                            print("\n Structured JSON response received:")
                            try:
                                parsed_json = json.loads(json_str)
                                print(json.dumps(parsed_json, indent=2, ensure_ascii=False))
                            except json.JSONDecodeError:
                                print("The response is not valid JSON after cleaning:", json_str)
                        else:
                            print("No JSON found in the API response:", cleaned_response)

                    except Exception as e:
                        print(f"Erro: '{product_name}': {e}")
            
        except Exception as e:
            print(f"Erro: {csv_file}: {e}")
            
        print("\n" + "="*60 + "\n")

⚙️ Processando arquivo do cliente: barbosa_monteiro_s.a..csv
   -> Encontrados produtos: ['Prolead descrição de cargos', 'Coremind (Sistema de gestão de conhecimento)', 'Guizo (Sistema de atendimento de clientes)', 'Prolead Express Yourself (Módulo para pesquisas)', 'Prolead PDI', 'PVP (Sistema de remuneração variável a curto prazo)']

   >> Analisando produto: 'Prolead descrição de cargos'...
      -> Calculando estatísticas com Pandas...
      -> Estatísticas encontradas:
- **Problemas de login:** 45% dos chamados
- **Exportação:** 31% dos chamados
- **Relatórios:** 24% dos chamados

   📲 Enviando prompt enriquecido para a API do Gemini...


E0000 00:00:1760148339.805758      43 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.



   ✅ Resposta JSON estruturada recebida:
{
  "company_name": "Barbosa Monteiro S.A.",
  "product_name": "Prolead descrição de cargos",
  "insights": [
    {
      "theme": "Problemas de login",
      "percentage": "45%",
      "actions": [
        "Revisar e otimizar o sistema de permissões e controle de acesso para garantir que usuários logados tenham as autorizações corretas para todas as funcionalidades (edição, salvamento, visualização de dados).",
        "Investigar e corrigir a causa raiz dos problemas de salvamento, carregamento e edição de cargos que ocorrem após o login ou após atualizações.",
        "Implementar mensagens de erro mais claras e informativas para falhas de acesso, permissão e operações de dados (salvar/carregar).",
        "Analisar logs de autenticação e autorização para identificar padrões de falha e diferenciar entre problemas de login inicial e questões de permissão pós-login."
      ]
    },
    {
      "theme": "Exportação",
      "percentage": "31%",
